# Invest Partner - Seu parceiro de Investimentos

### Imports e APIKey

In [1]:
import openai
from dotenv import load_dotenv, find_dotenv
import json
import yfinance as yf

In [2]:
_ = load_dotenv(find_dotenv())

client = openai.Client()

# Funções e Tools

In [3]:
def retorna_cotacao_acao_historica(
        ticker,
        periodo='1mo'
):
    
    ticker = yf.Ticker(ticker)
    hist = ticker.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')  # Formata o índice do DataFrame hist para strings no formato 'AAAA-MM-DD'.
    hist = round(hist, 2)   
    
    # Verifica se o DataFrame tem mais de 30 registros. Se tiver,  calcula um slice_size (tamanho fatia) dividindo por 30.
    # E reduz tamanho pegando um registro a cada slice_size, começando do final e revertendo para manter na sequência original.
    
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
    
    
    print("Função 1 foi chamada")
        
    return hist.to_json() # Passar como dicionario, e não como DF






def retorna_info(ticker):
    
    ticker = yf.Ticker(ticker)
    info = str(ticker.info)
        
        
    print("Função 2 foi chamada")
    return info




def retorna_metadados(ticker, periodo):
    
    ticker = yf.Ticker(ticker)
    hist = ticker.history(period=periodo)
    metadados = str(ticker.history_metadata)
        
        
    print("Função 3 foi chamada")
    return metadados





def retorna_noticias(ticker):
    ticker = yf.Ticker(ticker)
    noticias = str(ticker.news)
    
    return noticias




def retorna_desdobramentos(ticker):
    ticker = yf.Ticker(ticker)
    desdobramentos = ticker.splits
    
    return desdobramentos.to_json()



In [4]:
# Mapeando função
funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica,
                      'retorna_info': retorna_info,
                      'retorna_metadados': retorna_metadados,
                       'retorna_noticias': retorna_noticias,
                       'retorna_desdobramentos': retorna_desdobramentos
                      }

### Tools

In [5]:
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'retorna_cotacao_acao_historica',
            'description': 'Retorna a cotação diária histórica para uma ação da bovespa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históriocos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]  # API so aceita esses dias
                    }
                }
            }
        }
    },
    
    
    
    
    
     {
        'type': 'function',
        'function': {
            'name': 'retorna_info',
            'description': 'Retorna informações gerais sobre uma ação, incluindo uma variedade de dados, \
            como o nome da empresa, setor da indústria, descrição da empresa, país de origem, e mais. É útil para obter \
            uma visão geral rápida e detalhes básicos sobre a empresa associada ao ticker fornecido.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    }
                }
            }
        }
    },
    
    
    {
        'type': 'function',
        'function': {
            'name': 'retorna_metadados',
            'description': 'Fornece informações, de acordo com a data fornecida, sobre os dados históricos disponíveis para \
            uma ação, incluindo o intervalo de datas disponíveis, os tipos de preços incluídos  (como abertura, fechamento, \
            máximos, mínimos e volume), divisões de ações, ajustes de dividendos e outros eventos corporativos relevantes. \
            Essas informações são úteis para entender a qualidade e o escopo dos dados históricos disponíveis.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históriocos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]  # API so aceita esses dias
                    }
                }
            }
        }
    },
    
    

    
    
    
    
    {
        'type': 'function',
        'function': {
            'name': 'retorna_noticias',
            'description': 'retorna uma lista de notícias recentes relacionadas à empresa. Ele fornece manchetes, datas e links para artigos sobre a empresa cujas ações são negociadas na bolsa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    }
                }
            }
        }
    },
    
    
    {
        'type': 'function',
        'function': {
            'name': 'retorna_desdobramentos',
            'description': 'retorna uma série temporal contendo os históricos de desdobramentos (splits) de ações de uma empresa. Ele fornece as datas e as razões dos splits ocorridos ao longo do tempo para uma determinada ação listada na bolsa.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3.SA" para ambev, "PETR4.SA" para petrobras, etc'
                    }
                }
            }
        }
    }
    
    
]


### Geração de mensagens

In [6]:
def gera_resposta(mensagens):
    resposta = client.chat.completions.create(
        messages=mensagens,
        model="gpt-3.5-turbo-0125",
        max_tokens=1000,
        tools=tools,
        tool_choice='auto'
        #stream=True,
    )
    
    tool_calls = resposta.choices[0].message.tool_calls

    if tool_calls:
        mensagens.append(resposta.choices[0].message)
        for tool_call in tool_calls:
            func_name = tool_call.function.name
            function_to_call = funcoes_disponiveis[func_name]
            func_args = json.loads(tool_call.function.arguments)
            func_return = function_to_call(**func_args)  # Passa todos os parametros do 'func_args', para a function_to_call
            mensagens.append({
                'tool_call_id': tool_call.id,
                'role': 'tool',
                'name': func_name,
                'content': func_return
            })
            
        segunda_resposta = client.chat.completions.create(
            messages=mensagens,
            model="gpt-3.5-turbo-0125",   
        )
        mensagens.append(segunda_resposta.choices[0].message)
    
        print(f'Assistant: {mensagens[-1].content}') 
    
    
    
    
    
    
     # Responder dinamicamente, porem com problema na quebra de linha 
    
        '''
        print("Assistant: ", end='')   
    
        msg = mensagens[-1].content.split(" ")
    
        for caractere in msg:
            print(caractere, end='')
            print(" ", end='') 
            time.sleep(0.1)
        '''

        
    
    
    return mensagens


### Chat

In [ ]:
if __name__ == '__main__':

    print('Olá, Seja bem-vindo ao Invest Partner, o seu parceiro de Investimentos :)')
    mensagens = []
    while True:
        input_usuario = input('User: ')
        mensagens.append({'role': 'user', 'content': input_usuario})
        mensagens = gera_resposta(mensagens)

Olá, Seja bem-vindo ao Invest Partner, o seu parceiro de Investimentos :)
User: quero informações gerais Petr4, seja breve
Função 2 foi chamada
Assistant: PETR4, conhecida como Petrobras, é uma empresa brasileira do setor de petróleo e gás. Atua nas áreas de exploração, produção e venda de petróleo e gás no Brasil e internacionalmente. Possui cerca de 46.730 funcionários e sua sede fica no Rio de Janeiro. A Petrobras opera em três segmentos: Exploração e Produção; Refino, Transporte e Comercialização; e Gás e Energia. Seu ticker na bolsa de valores é PETR4.SA. O preço de fechamento anterior foi de R$ 37,18 e o preço atual é R$ 37,49. A empresa tem um dividend yield de 0,143% e um beta de 1,006. O valor de mercado é de R$ 495,9 bilhões e seu volume médio diário de negociação é de 49.481.809 ações. A Petrobras tem uma avaliação média de recomendação de compra por analistas e apresenta um potencial de alta no preço-alvo.
User: Quais as cotações dos ultimos 5 dias petr4?
Função 1 foi chama